# Model Analysis

#### Model Performance Accuracy (normalization + feature selection + uniform dataset)
    Linear Regression: 22.89%
    Logistic Regression: 89.93%
    SVM: 93.41%  
    NN: 96.25%
    KNN: Too many features for my device to compute
    
#### Kaggle Results
    SVM: 91.22%
    NN (original dataset): 94.81%
        (normalization): 96.58%
        (feature selection): 93.85%
        (normalization + feature selection): ?
    NN (uniform dataset): ?
        (normalization): _
        (normalization + feature selection): 95.18% 
    
#### Findings
    Normalization improves accuracy by ~2%
    Feature selection improves efficiency by ~10% but decreases accuracy by ~1%
    Optimization via shuffling the data can increase accuracy by ~1%
    
    TO DO: The uniform training set with only normalization will be the best
    
#### Hyperparameters for normalized NN
    Hidden Layer Activation: relu
    Solver: adam
    Alpha: 0.0001
    Learning rate: 0.001
    Shuffle: True
    Tolerence for Optimization : 0.0001
    Gradient Descent: 0.9
    Validation fraction: 0.1
    Loss fcn: 0.004860026848415791
    Iterations: 105
    Layers: 3 (1 hidden layer)
    Output activation: softmax

#### Notes
    By Uniform dataset I mean the labels in the dataset are a distributed uniformly

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

path = '/Users/antonmax2/Documents/dev/data/digit_recognizer/'
df_train = pd.read_csv(path+'train_data.csv')
df_val = pd.read_csv(path+'val_data.csv')
df_test = pd.read_csv(path+'test_data.csv')

In [126]:
# Converting dataframes -> numpy arrays to train models off of 
train = df_train.values
val = df_val.values
test = df_test.values

Xtr = train[:,1:]
Ytr = train[:,0]
Xval = val[:,1:]
Yval = val[:,0]
Xte = test[:,1:]
Yte = test[:,0]

In [3]:
# Normalizing data to use simple Keras models
from sklearn import preprocessing
Xtr = preprocessing.normalize(Xtr)
Xval = preprocessing.normalize(Xval)
Xte = preprocessing.normalize(Xte)

In [4]:
# Linear Regression with feature selection and normalization
from sklearn import linear_model
from sklearn.metrics import accuracy_score

regr = linear_model.LinearRegression()
regr.fit(Xtr, Ytr)
Yhat = np.floor(regr.predict(Xval))
print("Linear model accuracy: {0:.2f}%".format(accuracy_score(Yval, Yhat)*100))

/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


Linear model accuracy: 24.44%


In [5]:
# Logistic Regression with feature selection and normalization and uniform dataset
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(Xtr, Ytr)
Yhat = lr.predict(Xval)
print("Linear model accuracy: {0:.2f}%".format(accuracy_score(Yval, Yhat)*100))

Linear model accuracy: 89.93%


In [6]:
# SVM with feature selection and normalization and uniform dataset
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsOneClassifier

Yhat = OneVsOneClassifier(LinearSVC(random_state=0)).fit(Xtr, Ytr).predict(Xval)
print("SVM model accuracy: {0:.2f}%".format(accuracy_score(Yval, Yhat)*100))

SVM model accuracy: 93.41%


In [7]:
# NN with feature selection and normalization and uniform dataset
# Takes much longer than SVM
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier()
clf.fit(Xtr, Ytr)
Yhat = clf.predict(Xval)
print("NN model accuracy: {0:.2f}%".format(accuracy_score(Yval, Yhat)*100))

NN model accuracy: 96.17%


#### KNN takes too long on my device due to all the features present in this problem

In [15]:
# Normalization and feature selection and uniform dataset

path = '/Users/antonmax2/Documents/dev/data/digit_recognizer/'
test_model = pd.read_csv(path+'test.csv')

# Importing modified training data
path = '/Users/antonmax2/Documents/dev/data/digit_recognizer/'
train_model = pd.read_csv(path+'model_train.csv')

# Removing all features from test.csv not evaluated in our model
for i in range(784):
    col = "pixel" + str(i)
    if col not in train_model.columns:
        test_model = test_model.drop([col], axis=1)

In [60]:
# Kaggle SVM with normalization and feature selection and uniform dataset

Yhat = OneVsOneClassifier(LinearSVC(random_state=0)).fit(train_model.iloc[:,1:], train_model.iloc[:,0]).predict(test_model)

# Creating submission file
index = np.arange(1, Yhat.shape[0]+1)
submission = np.column_stack((index,Yhat))
np.savetxt('submission.txt', submission, delimiter=',', fmt='%i') # I add: "ImageId,Label" manually to first row

In [63]:
# Kaggle NN with normalization and feature selection and uniform dataset

Yhat = MLPClassifier().fit(train_model.iloc[:,1:], train_model.iloc[:,0]).predict(test_model)

index = np.arange(1, Yhat.shape[0]+1)
submission = np.column_stack((index,Yhat))
np.savetxt('submission.txt', submission, delimiter=',', fmt='%i')

In [70]:
# Kaggle NN with feature selection on original dataset

path = '/Users/antonmax2/Documents/dev/data/digit_recognizer/'
train_model = pd.read_csv(path+'train.csv')

# Removing all features from test.csv not evaluated in our model
for i in range(784):
    col = "pixel" + str(i)
    if col not in df_train.columns:
        train_model = train_model.drop([col], axis=1)
        
Yhat = MLPClassifier().fit(train_model.iloc[:,1:], train_model.iloc[:,0]).predict(test_model)

index = np.arange(1, Yhat.shape[0]+1)
submission = np.column_stack((index,Yhat))
np.savetxt('submission.txt', submission, delimiter=',', fmt='%i')

In [73]:
# Kaggle NN on original data

path = '/Users/antonmax2/Documents/dev/data/digit_recognizer/'
train_model = pd.read_csv(path+'train.csv')
path = '/Users/antonmax2/Documents/dev/data/digit_recognizer/'
test_model = pd.read_csv(path+'test.csv')

Yhat = MLPClassifier().fit(train_model.iloc[:,1:], train_model.iloc[:,0]).predict(test_model)

index = np.arange(1, Yhat.shape[0]+1)
submission = np.column_stack((index,Yhat))
np.savetxt('submission.txt', submission, delimiter=',', fmt='%i')

In [135]:
# Kaggle NN with normalized data on original data

path = '/Users/antonmax2/Documents/dev/data/digit_recognizer/'
train_model = pd.read_csv(path+'train.csv')
path = '/Users/antonmax2/Documents/dev/data/digit_recognizer/'
test_model = pd.read_csv(path+'test.csv')

labels = train_model.label.values
train_model = preprocessing.normalize(train_model.iloc[:,1:])

model = MLPClassifier().fit(train_model, labels)
Yhat = model.predict(test_model.values)

index = np.arange(1, Yhat.shape[0]+1)
submission = np.column_stack((index,Yhat))
np.savetxt('submission.txt', submission, delimiter=',', fmt='%i') # Note: Dataframes run quicker than numpy arrays on scikit

In [163]:
# Hyperparameters of NN above

print(model.activation)
print(model.solver)
print(model.alpha)
print(model.learning_rate_init)
print(model.shuffle)
print(model.tol)
print(model.momentum)
print(model.validation_fraction)
print(model.loss_)
print(model.n_iter_)
print(model.n_layers_)
print(model.out_activation_)

# Other hyperparameters
# print(model.batch_size) # batch_size=min(200, n_samples) # print(model.learning_rate) # print(model.power_t)
# print(model.max_iter) # print(model.random_state) # print(model.verbose) # print(model.warm_start)
# print(model.nesterovs_momentum) # print(model.early_stopping) # print(model.beta_1) # print(model.beta_2)
# print(model.epsilon) # print(model.classes_) # print(model.coefs_) # print(model.intercepts_) # print(model.n_outputs_)

relu
adam
0.0001
0.001
True
0.0001
0.9
0.1
0.004860026848415791
105
3
softmax


In [105]:
# Kaggle NN with normalized data on uniform data

path = '/Users/antonmax2/Documents/dev/data/digit_recognizer/'
train_model = pd.read_csv(path+'uniform.csv')
path = '/Users/antonmax2/Documents/dev/data/digit_recognizer/'
test_model = pd.read_csv(path+'test.csv')

labels = train_model.label.values
train_model = preprocessing.normalize(train_model.iloc[:,1:])

Yhat = MLPClassifier().fit(train_model, labels).predict(test_model.values)

index = np.arange(1, Yhat.shape[0]+1)
submission = np.column_stack((index,Yhat))
np.savetxt('submission.txt', submission, delimiter=',', fmt='%i') # Note: Dataframes run quicker than numpy arrays on scikit

In [ ]:
# Kaggle NN on uniform data

path = '/Users/antonmax2/Documents/dev/data/digit_recognizer/'
train_model = pd.read_csv(path+'train.csv')
path = '/Users/antonmax2/Documents/dev/data/digit_recognizer/'
test_model = pd.read_csv(path+'test.csv')

Yhat = MLPClassifier().fit(train_model.iloc[:,1:], train_model.iloc[:,0]).predict(test_model)

index = np.arange(1, Yhat.shape[0]+1)
submission = np.column_stack((index,Yhat)) ******************TODO
np.savetxt('submission.txt', submission, delimiter=',', fmt='%i')

In [ ]:
# Kaggle NN on uniform data with feature selection
********** TODO

In [ ]:
# Kaggle NN on original data with feature selection and normalization
********* TODO

In [133]:
# Testing effects of shuffling data

clf = MLPClassifier()
clf.fit(Xtr, Ytr)
Yhat = clf.predict(Xval)
print("NN model accuracy: {0:.2f}%".format(accuracy_score(Yval, Yhat)*100))

np.random.shuffle(train)
Xtr = train[:,1:]
Ytr = train[:,0]

clf = MLPClassifier()
clf.fit(Xtr, Ytr)
Yhat = clf.predict(Xval)
print("NN model accuracy: {0:.2f}%".format(accuracy_score(Yval, Yhat)*100))

NN model accuracy: 93.64%
NN model accuracy: 94.28%
